In [ ]:
from random import choices

a genetic representation of a solution

In [ ]:
genome = list[int] 
population = list[genome]

def generate_genome(length: int) -> genome:
    return choices([0, 1], k=length)

genome = generate_genome(N)
print (genome)